In [1]:
import ingenos
import numpy as np
import pandas as pd
import re
import matplotlib
import matplotlib.pyplot as plt

##### read in the data

In [2]:
base_dir = "../../../data/amplicon_sequencing/funestus/"

In [3]:
afun_csv = pd.read_csv(base_dir + "Afun_genotype_AS.csv", 
                       index_col = 0)
afun_csv.head()

,Sample,Raw Reads,On-Target Reads,%On-Target,%GT,IFI,2Ra_26087506,2Ra_26099532,2Ra_26111277,2Ra_26212609,...,3Rb_30123420,3Rb_30516692,3Rb_30661948,3Rb_30738354,3Rb_30739440,3Rb_30902012,3Rb_31397626,3Rb_31568791,3Rb_32435459,3Rb_32492194
0,GTseq_012_GTseq-i5_01_PLATE2_A01,279181,178545,63.95,97.78,0.20,11,11,11,11,...,12,11,11,12,11,11,11,11,11,11
1,GTseq_012_GTseq-i5_02_PLATE2_B01,153060,94210,61.55,96.67,0.24,11,11,11,11,...,11,11,11,0,11,11,11,11,11,11
2,GTseq_012_GTseq-i5_03_PLATE2_C01,81137,55467,68.36,95.56,0.46,11,0,11,11,...,11,11,11,0,11,11,11,11,11,11
3,GTseq_012_GTseq-i5_04_PLATE2_D01,179924,118383,65.80,83.33,0.44,22,0,22,22,...,12,0,12,0,11,12,12,0,12,11
4,GTseq_012_GTseq-i5_05_PLATE2_E01,162062,95472,58.91,94.44,0.29,11,0,11,11,...,11,11,11,0,11,11,11,11,11,11


In [4]:
afun_csv.shape

(235, 96)

##### reshape and trim as needed

In [5]:
column_list =\
[item[0] for item in afun_csv.set_index("Sample").iloc[:,5:].columns.str.split('_')]

trimmed = afun_csv.set_index("Sample").iloc[:,5:]

trimmed.reset_index(inplace=True)

IDs = [item[-2] + "_" + item[-1] for item in trimmed["Sample"].str.split("_")]

trimmed["Specimen"] = pd.Series(IDs)
trimmed["Specimen"] = trimmed["Specimen"].str.upper()

trimmed.drop(columns=["Sample"], inplace=True)
trimmed.set_index("Specimen", inplace=True)
trimmed = trimmed.T

trimmed["label"] = column_list

trimmed["assay"] = trimmed.index

trimmed.head()

Specimen,PLATE2_A01,PLATE2_B01,PLATE2_C01,PLATE2_D01,PLATE2_E01,PLATE2_F01,PLATE2_G01,PLATE2_H01,PLATE2_A02,PLATE2_B02,...,PLATE3_C10,PLATE3_D10,PLATE3_A11,PLATE3_B11,PLATE3_C11,PLATE3_A12,PLATE3_B12,PLATE3_C12,label,assay
2Ra_26087506,11,11,11,22,11,11,11,11,11,11,...,22,11,0,22,11,22,11,11,2Ra,2Ra_26087506
2Ra_26099532,11,11,0,0,0,0,0,11,11,11,...,11,0,0,0,11,0,0,0,2Ra,2Ra_26099532
2Ra_26111277,11,11,11,22,11,11,11,11,0,11,...,12,11,22,22,11,22,11,11,2Ra,2Ra_26111277
2Ra_26212609,11,11,11,22,11,11,11,11,11,11,...,12,11,12,22,11,22,12,11,2Ra,2Ra_26212609
2Ra_26213556,11,11,11,22,11,11,11,11,11,11,...,12,11,12,22,11,22,12,11,2Ra,2Ra_26213556


In [6]:
trimmed = trimmed.reset_index(drop=True).set_index(["label", "assay"]).T
trimmed.head()

label               2Ra                                                      \
assay      2Ra_26087506 2Ra_26099532 2Ra_26111277 2Ra_26212609 2Ra_26213556   
Specimen                                                                      
PLATE2_A01           11           11           11           11           11   
PLATE2_B01           11           11           11           11           11   
PLATE2_C01           11            0           11           11           11   
PLATE2_D01           22            0           22           22           22   
PLATE2_E01           11            0           11           11           11   

label                                                                        \
assay      2Ra_26558370 2Ra_26854967 2Ra_26856512 2Ra_26945231 2Ra_27077779   
Specimen                                                                      
PLATE2_A01           11           11           11           11           11   
PLATE2_B01           11           11           11           11           11   
PLATE2_C01           11           11           11           11           11   
PLATE2_D01           22            0           22            0           22   
PLATE2_E01           11           11           11           11           11   

label       ...          3Rb                                         \
assay       ... 3Rb_30123420 3Rb_30516692 3Rb_30661948 3Rb_30738354   
Specimen    ...                                                       
PLATE2_A01  ...           12           11           11           12   
PLATE2_B01  ...           11           11           11            0   
PLATE2_C01  ...           11           11           11            0   
PLATE2_D01  ...           12            0           12            0   
PLATE2_E01  ...           11           11           11            0   

label                                                                        \
assay      3Rb_30739440 3Rb_30902012 3Rb_31397626 3Rb_31568791 3Rb_32435459   
Specimen                                                                      
PLATE2_A01           11           11           11           11           11   
PLATE2_B01           11           11           11           11           11   
PLATE2_C01           11           11           11           11           11   
PLATE2_D01           11           12           12            0           12   
PLATE2_E01           11           11           11           11           11   

label                    
assay      3Rb_32492194  
Specimen                 
PLATE2_A01           11  
PLATE2_B01           11  
PLATE2_C01           11  
PLATE2_D01           11  
PLATE2_E01           11  

[5 rows x 90 columns]

##### read in the metadata

In [7]:
cyto = pd.read_excel(base_dir + "AFUN_ALL_3_PLATES_11Feb19_v2.xlsx", 
                     index_col = False)

cyto.head()

,Plate name,Row,Column,volume,conc,SpecimenID,Date of collection,Year,Season,Month,...,X_2S,X_3A,X_3B,X_5A,2RAS,Kariotype,chormosomal form,READER,CSP,Blood meal
0,PLATE_01,A,1,25ul,NaN,00-00021,2000-01-04,2000.0,99-00,1.0,...,0,0,0,0,00,00000,Kiribina,Moussa,carcass for ND,NaN
1,PLATE_01,A,2,25ul,NaN,00-00022,2000-01-04,2000.0,99-00,1.0,...,0,0,0,0,00,00000,Kiribina,Moussa,carcass for ND,NaN
2,PLATE_01,A,3,25ul,NaN,00-00024,2000-01-04,2000.0,99-00,1.0,...,1,1,0,0,01,01100,Kiribina,Moussa,carcass for ND,NaN
3,PLATE_01,A,4,25ul,NaN,00-00025,2000-01-04,2000.0,99-00,1.0,...,0,1,2,0,00,00120,Folonzo,Moussa,carcass for ND,NaN
4,PLATE_01,A,5,25ul,NaN,00-00029,2000-01-04,2000.0,99-00,1.0,...,0,0,0,0,00,00000,Kiribina,Moussa,carcass for ND,NaN


In [8]:
cyto.columns

Index(['Plate name', 'Row', 'Column', 'volume', 'conc', 'SpecimenID',
       'Date of collection', 'Year', 'Season', 'Month', 'Village', 'Method',
       'Sample', 'Time-slot', 'Bait_name', 'Position', 'Compound', 'X_2A',
       'X_2S', 'X_3A', 'X_3B', 'X_5A', '2RAS', 'Kariotype', 'chormosomal form',
       'READER', 'CSP', 'Blood meal'],
      dtype='object')

In [9]:
cyto = cyto[["Plate name", "Row", "Column", "SpecimenID", "X_2A", "X_3A", "X_3B"]]

cyto.head()

,Plate name,Row,Column,SpecimenID,X_2A,X_3A,X_3B
0,PLATE_01,A,1,00-00021,0,0,0
1,PLATE_01,A,2,00-00022,0,0,0
2,PLATE_01,A,3,00-00024,0,1,0
3,PLATE_01,A,4,00-00025,0,1,2
4,PLATE_01,A,5,00-00029,0,0,0


In [10]:
cyto.rename(columns = {"X_2A" : "2Ra",
                      "X_3A" : "3Ra",
                      "X_3B" : "3Rb",
                      "SpecimenID" : "ID"}, inplace=True)

##### narrow to those rows corresponding to wells that were used

In [11]:
np.sum(cyto["ID"].isnull())

50

In [12]:
cyto = cyto.loc[~(cyto["ID"].isnull()), :]

##### match ID format to that found in the data

In [13]:
trimmed.head()

label               2Ra                                                      \
assay      2Ra_26087506 2Ra_26099532 2Ra_26111277 2Ra_26212609 2Ra_26213556   
Specimen                                                                      
PLATE2_A01           11           11           11           11           11   
PLATE2_B01           11           11           11           11           11   
PLATE2_C01           11            0           11           11           11   
PLATE2_D01           22            0           22           22           22   
PLATE2_E01           11            0           11           11           11   

label                                                                        \
assay      2Ra_26558370 2Ra_26854967 2Ra_26856512 2Ra_26945231 2Ra_27077779   
Specimen                                                                      
PLATE2_A01           11           11           11           11           11   
PLATE2_B01           11           11           11           11           11   
PLATE2_C01           11           11           11           11           11   
PLATE2_D01           22            0           22            0           22   
PLATE2_E01           11           11           11           11           11   

label       ...          3Rb                                         \
assay       ... 3Rb_30123420 3Rb_30516692 3Rb_30661948 3Rb_30738354   
Specimen    ...                                                       
PLATE2_A01  ...           12           11           11           12   
PLATE2_B01  ...           11           11           11            0   
PLATE2_C01  ...           11           11           11            0   
PLATE2_D01  ...           12            0           12            0   
PLATE2_E01  ...           11           11           11            0   

label                                                                        \
assay      3Rb_30739440 3Rb_30902012 3Rb_31397626 3Rb_31568791 3Rb_32435459   
Specimen                                                                      
PLATE2_A01           11           11           11           11           11   
PLATE2_B01           11           11           11           11           11   
PLATE2_C01           11           11           11           11           11   
PLATE2_D01           11           12           12            0           12   
PLATE2_E01           11           11           11           11           11   

label                    
assay      3Rb_32492194  
Specimen                 
PLATE2_A01           11  
PLATE2_B01           11  
PLATE2_C01           11  
PLATE2_D01           11  
PLATE2_E01           11  

[5 rows x 90 columns]

In [14]:
IDs[:5]

['PLATE2_A01', 'PLATE2_B01', 'PLATE2_C01', 'PLATE2_D01', 'PLATE2_E01']

In [15]:
cyto["Specimen"] = cyto["Plate name"].replace(
    regex=True, to_replace="_0", value="").replace(
    regex=True, to_replace="_", value="") +\
"_" + cyto["Row"].map(str) + cyto["Column"].astype(str).str.zfill(2)

cyto.head()

,Plate name,Row,Column,ID,2Ra,3Ra,3Rb,Specimen
0,PLATE_01,A,1,00-00021,0,0,0,PLATE1_A01
1,PLATE_01,A,2,00-00022,0,0,0,PLATE1_A02
2,PLATE_01,A,3,00-00024,0,1,0,PLATE1_A03
3,PLATE_01,A,4,00-00025,0,1,2,PLATE1_A04
4,PLATE_01,A,5,00-00029,0,0,0,PLATE1_A05


##### check the specimen that had an incorrect Specimen label in another version of the metadata

In [16]:
cyto.loc[cyto["ID"] == "01-01035"]

,Plate name,Row,Column,ID,2Ra,3Ra,3Rb,Specimen
168,PLATE_02,G,1,01-01035,0,0,0,PLATE2_G01


##### in the results, replace genotypes as generated by GTseq with counts of alternate alleles

In [17]:
trimmed.replace(to_replace={0 : np.nan,
            11 : 0,
            12 : 1,
            22 : 2}, value=None, inplace=True)

##### check that the empty wells were omitted

In [18]:
trimmed.index.contains("PLATE1_A01")

False

In [19]:
trimmed.index.contains("PLATE1_G01")

False

In [20]:
trimmed.index.contains("PLATE1_H01")

False

##### examine what proportion of calls are missing per SNP and per specimen

In [21]:
inversions = trimmed.columns.levels[0]

inversions

Index(['2Ra', '3Ra', '3Rb'], dtype='object', name='label')

In [22]:
missing_df = pd.DataFrame(columns = ["label", "n_called"])

for inversion in inversions:
    
    chunk = trimmed.loc[:,inversion]

    n_specs = chunk.shape[0]
    n_snps = chunk.shape[1]
    
    missing_df = missing_df.append(pd.DataFrame({"label" : [inversion] * n_snps,
                 "n_called" : ((1-(np.sum(chunk.isnull(), axis=0) / n_specs)) * 100).values}))

missing_df.head()

,label,n_called
0,2Ra,91.063830
1,2Ra,45.106383
2,2Ra,96.595745
3,2Ra,98.297872
4,2Ra,97.872340


In [23]:
missing_df.reset_index(inplace=True, drop=True)
missing_df.drop_duplicates(subset="label").index

Int64Index([0, 29, 62], dtype='int64')

In [24]:
trimmed["2Ra"].columns[(trimmed["2Ra"].isnull().mean() > 0.2).values].values

array(['2Ra_26099532', '2Ra_27094835', '2Ra_28137208', '2Ra_32237544'],
      dtype=object)

In [25]:
trimmed["3Ra"].columns[(trimmed["3Ra"].isnull().mean() > 0.2).values]

Index(['3Ra_10957820', '3Ra_11012511'], dtype='object', name='assay')

In [26]:
trimmed["3Rb"].columns[(trimmed["3Rb"].isnull().mean() > 0.2).values]

Index(['3Rb_25434979', '3Rb_30738354'], dtype='object', name='assay')

##### drop these

In [27]:
drop = {}

drop["2Ra"] = trimmed["2Ra"].columns[(trimmed["2Ra"].isnull().mean() > 0.2).values].values

drop["3Ra"] = trimmed["3Ra"].columns[(trimmed["3Ra"].isnull().mean() > 0.2).values].values

drop["3Rb"] = trimmed["3Rb"].columns[(trimmed["3Rb"].isnull().mean() > 0.2).values].values

In [28]:
trimmed.shape

(235, 90)

In [29]:
trimmed.drop(columns=drop["2Ra"], level=1, axis=1, inplace=True)
trimmed.drop(columns=drop["3Ra"], level=1, axis=1, inplace=True)
trimmed.drop(columns=drop["3Rb"], level=1, axis=1, inplace=True)

trimmed.shape

(235, 82)

##### call rates by specimen

In [30]:
sample_missingness = pd.DataFrame(trimmed.isnull().mean(axis=1))
sample_missingness.columns = ["null_rate"]
sample_missingness.head()

,null_rate
Specimen,
PLATE2_A01,0.000000
PLATE2_B01,0.000000
PLATE2_C01,0.000000
PLATE2_D01,0.085366
PLATE2_E01,0.000000


In [31]:
sample_missingness[sample_missingness["null_rate"] > 0.2]

,null_rate
Specimen,
PLATE2_G01,0.207317
PLATE2_D09,1.000000
PLATE1_B07,1.000000
PLATE1_H10,0.378049
PLATE1_E12,0.378049


##### which specimens?

In [32]:
samples_low_call = sample_missingness[sample_missingness["null_rate"] > 0.2].index.values

In [33]:
cyto.loc[cyto["Specimen"].isin(samples_low_call)]

,Plate name,Row,Column,ID,2Ra,3Ra,3Rb,Specimen
18,PLATE_01,B,7,00-00087,0,0,0,PLATE1_B07
59,PLATE_01,E,12,00-00263,0,0,0,PLATE1_E12
91,PLATE_01,H,10,00-02985,#N/R,2,1,PLATE1_H10
140,PLATE_02,D,9,01-00456,1,2,1,PLATE2_D09
168,PLATE_02,G,1,01-01035,0,0,0,PLATE2_G01


In [34]:
trimmed.shape

(235, 82)

In [35]:
trimmed.drop(index=samples_low_call).shape

(230, 82)

In [36]:
trimmed.drop(index=samples_low_call, inplace=True)

##### remove these specimens from the metadata

In [37]:
cyto.head()

,Plate name,Row,Column,ID,2Ra,3Ra,3Rb,Specimen
0,PLATE_01,A,1,00-00021,0,0,0,PLATE1_A01
1,PLATE_01,A,2,00-00022,0,0,0,PLATE1_A02
2,PLATE_01,A,3,00-00024,0,1,0,PLATE1_A03
3,PLATE_01,A,4,00-00025,0,1,2,PLATE1_A04
4,PLATE_01,A,5,00-00029,0,0,0,PLATE1_A05


In [38]:
cyto.shape

(238, 8)

In [39]:
cyto = cyto.loc[(~cyto["Specimen"].isin(samples_low_call)), :]

cyto.shape

(233, 8)

In [40]:
trimmed.shape

(230, 82)

##### also drop the three unused wells

In [41]:
cyto = cyto.loc[(~(cyto["Specimen"].isin(["PLATE1_A01", "PLATE1_G01", "PLATE1_H01"]))), :]

In [42]:
cyto.shape

(230, 8)

In [43]:
trimmed.columns = trimmed.columns.remove_unused_levels()
trimmed.columns.shape

(82,)

In [44]:
trimmed.index.shape

(230,)

In [45]:
for inversion in trimmed.columns.levels[0]:
    
    print(inversion)
    print(trimmed[inversion].shape)

2Ra
(230, 25)
3Ra
(230, 31)
3Rb
(230, 26)


##### identify inversion genotypes

In [46]:
results = pd.DataFrame(index = trimmed.index, 
                       columns = [name + "_mean" for\
                                  name in trimmed.columns.levels[0]])

results.head()

,2Ra_mean,3Ra_mean,3Rb_mean
Specimen,,,
PLATE2_A01,NaN,NaN,NaN
PLATE2_B01,NaN,NaN,NaN
PLATE2_C01,NaN,NaN,NaN
PLATE2_D01,NaN,NaN,NaN
PLATE2_E01,NaN,NaN,NaN


In [47]:
for inversion in inversions:
    
    chunk = trimmed[inversion]
    
    for index, row in chunk.iterrows():
        
        results.loc[index, inversion + "_mean"] = np.mean(row)

In [48]:
for column in results.columns:
    
    new_name = re.sub("_mean", "", column) + "_assigned"
    
    karyos = []
    
    for value in results[column].values:
        
        if value <= (2/3):
            
            karyos.append(0)
            
        elif value > (2/3) and value <= (4/3):
            
            karyos.append(1)
            
        else:
            
            karyos.append(2)
            
    results[new_name] = karyos

In [49]:
results.head()

,2Ra_mean,3Ra_mean,3Rb_mean,2Ra_assigned,3Ra_assigned,3Rb_assigned
Specimen,,,,,,
PLATE2_A01,0.04,0.0322581,0.0769231,0,0,0
PLATE2_B01,0.04,0,0.0384615,0,0,0
PLATE2_C01,0,0,0.0769231,0,0,0
PLATE2_D01,1.90909,2,0.652174,2,2,0
PLATE2_E01,0,0,0,0,0,0


In [50]:
results["2Ra_assigned"].value_counts()

0    164
2     33
1     33
Name: 2Ra_assigned, dtype: int64

In [51]:
results["3Ra_assigned"].value_counts()

0    132
2     64
1     34
Name: 3Ra_assigned, dtype: int64

In [52]:
results["3Rb_assigned"].value_counts()

0    166
1     37
2     27
Name: 3Rb_assigned, dtype: int64

##### merge with cytogenetic data

In [53]:
results.shape

(230, 6)

In [54]:
cyto.shape

(230, 8)

In [55]:
compare = pd.merge(results.reset_index(), cyto, on="Specimen")
compare.head()

,Specimen,2Ra_mean,3Ra_mean,3Rb_mean,2Ra_assigned,3Ra_assigned,3Rb_assigned,Plate name,Row,Column,ID,2Ra,3Ra,3Rb
0,PLATE2_A01,0.04,0.0322581,0.0769231,0,0,0,PLATE_02,A,1,00-02989,#N/R,2,0
1,PLATE2_B01,0.04,0,0.0384615,0,0,0,PLATE_02,B,1,01-00211,0,0,0
2,PLATE2_C01,0,0,0.0769231,0,0,0,PLATE_02,C,1,01-00282,0,0,0
3,PLATE2_D01,1.90909,2,0.652174,2,2,0,PLATE_02,D,1,01-00300,1,2,1
4,PLATE2_E01,0,0,0,0,0,0,PLATE_02,E,1,01-00512,1,0,0


In [56]:
compare.rename(columns = {"2Ra" : "2Ra_cyto",
                         "3Ra" : "3Ra_cyto",
                         "3Rb" : "3Rb_cyto"}, inplace=True)

compare = compare[["Specimen", "ID", "2Ra_mean", "3Ra_mean", "3Rb_mean", 
                   "2Ra_assigned", "3Ra_assigned", "3Rb_assigned",
                   "2Ra_cyto", "3Ra_cyto", "3Rb_cyto"]]

compare.head()

,Specimen,ID,2Ra_mean,3Ra_mean,3Rb_mean,2Ra_assigned,3Ra_assigned,3Rb_assigned,2Ra_cyto,3Ra_cyto,3Rb_cyto
0,PLATE2_A01,00-02989,0.04,0.0322581,0.0769231,0,0,0,#N/R,2,0
1,PLATE2_B01,01-00211,0.04,0,0.0384615,0,0,0,0,0,0
2,PLATE2_C01,01-00282,0,0,0.0769231,0,0,0,0,0,0
3,PLATE2_D01,01-00300,1.90909,2,0.652174,2,2,0,1,2,1
4,PLATE2_E01,01-00512,0,0,0,0,0,0,1,0,0
